In [ ]:
from pathlib import Path

if not Path('colab_utils.py').exists():
    !wget https://raw.githubusercontent.com/gschramm/parallelproj/master/notebooks/notebook_utils_parallelproj.py

In [ ]:
# check if we are running in google colab
# if we are running in colab, we install conda/mamba

import sys
import subprocess
from colab_utils import in_colab

if in_colab():
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'condacolab'])
    import condacolab
    condacolab.install()

 # the kernel gets restarted here (after conda install)
 # so all variables are deleted if you run this cell

In [ ]:
# if we we are running in colab, we install parallelproj from conda-forge
# and set the environment variable for the parallelproj c/cuda library
# we need to redo the check for COLAB, because the install
# of conda on colab, restarts the kernel and deletes all variables
import os
import subprocess
from colab_utils import in_colab
from distutils.spawn import find_executable
CUDA_PRESENT = find_executable('nvidia-smi') is not None 

if in_colab():
    subprocess.run(['mamba', 'install', 'parallelproj'])
    os.environ['PARALLELPROJ_C_LIB']='/usr/local/lib/libparallelproj_c.so'
    if CUDA_PRESENT:
        os.environ['PARALLELPROJ_CUDA_LIB']='/usr/local/lib/libparallelproj_cuda.so'
        subprocess.run(['mamba', 'install', 'cupy'])

print(f'notebook running in google colab: {in_colab()}')
print(f'cuda present: {CUDA_PRESENT}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import parallelproj
from notebook_utils_parallelproj import tonumpy

if parallelproj.cupy_enabled:
    import cupy as cp
    import cupy as xp
else:
    import numpy as xp

In [ ]:
# setup a test image

# image dimensions
n0, n1, n2 = (1, 128, 128)
img_dim = (n0, n1, n2)

# voxel size of the image
voxel_size = xp.array([2., 2., 2.]).astype(xp.float32)

# image origin -> world coordinates of the [0,0,0] voxel
img_origin = ((-xp.array(img_dim) / 2 + 0.5) * voxel_size).astype(xp.float32)

# setup a random image
img = xp.zeros((n0,n1,n2)).astype(xp.float32)
img[:,(n1//4):((3*n1)//4), (n2//4):((3*n2)//4)] = 1
img[:,(n1//4):(n1//3), (n2//4):(n2//3)] = 3


In [ ]:
# setup the coordinates for projections along parallel views
num_rad = 223
num_phi = 190

# radial coordinates of the projection views in mm
r = xp.linspace(-200, 200, num_rad, dtype = xp.float32)
# "radius" of the scanner in mm
R = 350.

projector = ParallelViewProjector2D(img_dim, r, num_phi, R, img_origin, voxel_size)

fig = projector.show_views(image = img, cmap = 'Greys')



In [ ]:
# do a non-TOF forward projection
img_fwd = projector(img)

In [ ]:
# do a non-TOF back projection (the adjoint of the forward projection)
img_back = projector.adjoint(img_fwd)


In [ ]:
im_kwargs = dict(origin = 'lower', cmap = 'Greys')

fig2, ax2 = plt.subplots(1,3,figsize=(12,4))
im0 = ax2[0].imshow(tonumpy(img[0,...]).T, **im_kwargs)
im1 = ax2[1].imshow(tonumpy(img_fwd), cmap = 'Greys')
im2 = ax2[2].imshow(tonumpy(img_back[0,...]).T, **im_kwargs)
ax2[0].set_title('image - x', fontsize = 'small')
ax2[1].set_title('forward projection - Ax', fontsize = 'small')
ax2[2].set_title('back projection of forward projection - A^HAx', fontsize = 'small'
)
cb0 = fig.colorbar(im0, fraction = 0.03)
cb1 = fig.colorbar(im1, fraction = 0.03)
cb2 = fig.colorbar(im2, fraction = 0.03)

ax2[0].set_xlabel('x1')
ax2[0].set_ylabel('x2')

ax2[1].set_xlabel('radial bin')
ax2[1].set_ylabel('view number')

ax2[2].set_xlabel('x1')
ax2[2].set_ylabel('x2')

fig2.tight_layout()